In [3]:
import numpy as np
import pandas as pd
import IPython.display
from six.moves import cPickle as pickle
from tqdm import tqdm
tqdm.pandas()
from IPython.display import display


def maybe_pickle(file_name, load_dataset, force=False):
    pickle_file_name = "pickle/2_" + file_name + ".pickle"
    import os
    if not os.path.exists("pickle"):
        os.makedirs("pickle")
        
    if os.path.exists(pickle_file_name) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping pickling.' % pickle_file_name)
    else:
        print('Pickling %s.' % pickle_file_name)
        dataset = load_dataset(None)
        try:
            with open(pickle_file_name, 'wb') as f:
                pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to', file_name, ':', e)
    
    return pickle_file_name

def load_data(file_name, force=False):
    original_file_path = "../input/" + file_name + ".csv"
    pickle_file_name = maybe_pickle(file_name, lambda x: pd.read_csv(original_file_path), force)
    
    with open(pickle_file_name, 'rb') as f:
        return pickle.load(f)

In [4]:
biology = load_data("biology")
cooking = load_data("cooking")
crypto = load_data("crypto")
diy = load_data("diy")
robotics = load_data("robotics")
travel = load_data("travel")

Pickling pickle/2_biology.pickle.
Pickling pickle/2_cooking.pickle.
Pickling pickle/2_crypto.pickle.
Pickling pickle/2_diy.pickle.
Pickling pickle/2_robotics.pickle.
Pickling pickle/2_travel.pickle.


In [81]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
stemmer = SnowballStemmer("english")
wordnet_lemmatizer = WordNetLemmatizer()
punctuation_trans_table = str.maketrans({key: None for key in string.punctuation})
html_tag_regex = re.compile('<.*?>')
code_tag_regex = re.compile('<code>([^<]+)</code>', re.S)
a_tag_regex = re.compile('<a href([^<]+)</a>', re.S)

def cleaning_text(text):
    original_text_length = len(text)
    number_of_html_tag = len(re.findall(html_tag_regex, text))
    number_of_code_fragments = len(re.findall(code_tag_regex, text))
    number_of_a_href = len(re.findall(a_tag_regex, text))
    
    # convert to lowercase
    text = text.lower()
    # remove code fragment
    text = re.sub(code_tag_regex, 'code_tag', text)
    # remove html tags
    text = re.sub(html_tag_regex, '', text)
    # remove \r, \n
    text = text.replace('\n', ' ').replace('\r', '')
    # remove Punctuations
    text = text.translate(punctuation_trans_table)
    # split
    words = word_tokenize(text)
    # remove stop words
    words = [word for word in words if word not in stopwords.words('english')]
    # lemmatizing, stemming
    #words = [wordnet_lemmatizer.lemmatize(word) for word in words]
    #words = [stemmer.stem(word) for word in words]
    # join
    text = ' '.join(words)
    
    number_of_cleaned_text_tokens = len(words)
    cleaned_text_length = len(text)
    return text, [original_text_length, number_of_html_tag, number_of_code_fragments, number_of_a_href, number_of_cleaned_text_tokens, cleaned_text_length]


def cleaning(row):
    row['title'], title_meta_list = cleaning_text(row['title'])
    row['title_original_text_length'] = title_meta_list[0]
    row['title_number_of_cleaned_text_tokens'] = title_meta_list[4]
    row['title_cleaned_text_length'] = title_meta_list[5]
    
    row['content'], content_meta_list = cleaning_text(row['content'])
    row['content_original_text_length'] = content_meta_list[0]
    row['content_number_of_html_tag'] = content_meta_list[1]
    row['content_number_of_code_fragments'] = content_meta_list[2]
    row['content_number_of_a_href'] = content_meta_list[3]
    row['content_number_of_cleaned_text_tokens'] = content_meta_list[4]
    row['content_cleaned_text_length'] = content_meta_list[5]
    
    row['cleaned_tags'], content_meta_list = cleaning_text(row['tags'])
    return row

In [82]:
def load_cleaned_df(file_name, force=False):
    original_file_path = "../input/" + file_name + ".csv"
    df = pd.read_csv(original_file_path)
    print("total len : %d" % len(df))
    result_df = df.progress_apply(cleaning, axis=1)
    
    # feature scaling for meta columns
    from sklearn import preprocessing
    min_max_scaler = preprocessing.MinMaxScaler()
    result_df['title_original_text_length'] = min_max_scaler.fit_transform(result_df['title_original_text_length'])
    result_df['title_number_of_cleaned_text_tokens'] = min_max_scaler.fit_transform(result_df['title_number_of_cleaned_text_tokens'])
    result_df['title_cleaned_text_length'] = min_max_scaler.fit_transform(result_df['title_cleaned_text_length'])
    result_df['content_original_text_length'] = min_max_scaler.fit_transform(result_df['content_original_text_length'])
    result_df['content_number_of_html_tag'] = min_max_scaler.fit_transform(result_df['content_number_of_html_tag'])
    result_df['content_number_of_code_fragments'] = min_max_scaler.fit_transform(result_df['content_number_of_code_fragments'])
    result_df['content_number_of_a_href'] = min_max_scaler.fit_transform(result_df['content_number_of_a_href'])
    result_df['content_number_of_cleaned_text_tokens'] = min_max_scaler.fit_transform(result_df['content_number_of_cleaned_text_tokens'])
    result_df['content_cleaned_text_length'] = min_max_scaler.fit_transform(result_df['content_cleaned_text_length'])
    
    return result_df
    
def maybe_pickle_cleaned_df(file_name, force=False):
    pickle_file_name = maybe_pickle(file_name + "_cleaned", lambda x: load_cleaned_df(file_name), force)
    
    with open(pickle_file_name, 'rb') as f:
        return pickle.load(f)

In [83]:
biology_cleaned_df = maybe_pickle_cleaned_df('biology', True)
cooking_cleaned_df = maybe_pickle_cleaned_df('cooking', True)
crypto_cleaned_df = maybe_pickle_cleaned_df('crypto', True)
diy_cleaned_df = maybe_pickle_cleaned_df('diy', True)
robotics_cleaned_df = maybe_pickle_cleaned_df('robotics', True)
travel_cleaned_df = maybe_pickle_cleaned_df('travel', True)

Pickling pickle/2_biology_cleaned.pickle.
total len : 13196


13197it [07:25, 29.61it/s]
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, Depreca

Pickling pickle/2_cooking_cleaned.pickle.
total len : 15404


15405it [08:19, 30.82it/s]
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data ei

Pickling pickle/2_crypto_cleaned.pickle.
total len : 10432


...

25919it [17:43, 24.36it/s]
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data ei

Pickling pickle/2_robotics_cleaned.pickle.
total len : 2771


2772it [02:13, 20.74it/s]
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data eit

Pickling pickle/2_travel_cleaned.pickle.
total len : 19279


19280it [11:59, 25.80it/s]
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\develop\Anaconda3\envs\python35\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data ei

In [84]:
full_df = pd.concat([biology_cleaned_df, cooking_cleaned_df, crypto_cleaned_df, diy_cleaned_df, robotics_cleaned_df, travel_cleaned_df])

In [85]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
stop_words = text.ENGLISH_STOP_WORDS

full_df_vectorizer = TfidfVectorizer(stop_words=stop_words)
full_df_vectors = full_df_vectorizer.fit_transform((full_df['title'] + " " + full_df['content']).tolist())
print(len(full_df_vectorizer.get_feature_names()))

151721


In [86]:
full_df_title_vectorizer = TfidfVectorizer(stop_words=stop_words, max_df=0.95, min_df=2,max_features=4000)
full_df_title_vectors = full_df_title_vectorizer.fit_transform((full_df['title']).tolist())
print(len(full_df_title_vectorizer.get_feature_names()))

4000


In [87]:
# code from https://buhrmann.github.io/tfidf-analysis.html

def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids if row[i] > 0.3]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)

def predict_tags(vectors, vectorizer, index):
    tfidf_df = top_feats_in_doc(vectors, vectorizer.get_feature_names(), index)
    return ' '.join(tfidf_df['feature'])


for i in range(0,10):
    print(predict_tags(full_df_title_vectors, full_df_title_vectorizer, i))
    print(full_df[i:i+1]['cleaned_tags'])

codon translation relative binding site start
0    ribosome bindingsites translation syntheticbio...
Name: cleaned_tags, dtype: object
experiments rna based
1    rna biochemistry
Name: cleaned_tags, dtype: object
groups sizes
2    immunology cellbiology hematology
Name: cleaned_tags, dtype: object
lb maintain selection good long
3    cellculture
Name: cleaned_tags, dtype: object
order
4    splicing mrna spliceosome introns exons
Name: cleaned_tags, dtype: object
avoid dna
5    dna biochemistry molecularbiology
Name: cleaned_tags, dtype: object
conditions form
6    neuroscience synapses
Name: cleaned_tags, dtype: object
ship
7    plasmids
Name: cleaned_tags, dtype: object
gene choosing reason
8    moleculargenetics geneexpression experimentald...
Name: cleaned_tags, dtype: object
occur times
9    evolution mitochondria chloroplasts
Name: cleaned_tags, dtype: object
